In [0]:
import sys
sys.version

'3.6.7 (default, Oct 22 2018, 11:32:17) \n[GCC 8.2.0]'

In [0]:
# !pip install tensorflow==1.12.0
# !pip install paramiko
import tensorflow
print(tensorflow.keras.__version__)
print(tensorflow.__version__)


2.2.4-tf
1.13.1


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········


In [0]:
import pandas
import numpy as np
import pickle  
import time  
import json
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import backend as K
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, Bidirectional, Activation, LSTM,Dropout, InputLayer
from gensim.models import KeyedVectors

In [0]:
# from keras.models import model_from_json

df = pandas.read_table('/content/gdrive/My Drive/yetty/data/all/onegram_input.utf8', header=None)
label = pandas.read_table('/content/gdrive/My Drive/yetty/data/all/onegram_label.txt', header=None)

In [0]:
df['label'] = label[0]
df.columns = ['character', 'label']

In [0]:
df = df.sample(50000)
df.sample(5)

,character,label
2854408,，,S
14390050,、,S
13981445,健,B
9419187,济,E
4643829,主,E


In [0]:
### Create sequence

vocabulary_size = 128

tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(df['character'])
sequences = tokenizer.texts_to_sequences(df['character'])
data = pad_sequences(sequences, maxlen=50)

data.shape

(50000, 50)

In [0]:
labels = LabelEncoder().fit_transform(df.label)
labels[:54]

array([0, 1, 3, 1, 1, 0, 1, 3, 3, 0, 3, 2, 3, 0, 1, 3, 3, 1, 1, 0, 1, 0,
       0, 1, 3, 3, 0, 3, 1, 1, 1, 3, 2, 2, 1, 0, 3, 1, 1, 1, 1, 1, 0, 2,
       0, 1, 3, 1, 1, 2, 0, 1, 3, 0])

In [0]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.33, random_state=42)

In [0]:
X_train.shape, y_train.shape

((33500, 50), (33500,))

In [0]:
####Load Embedding
embedding_model = KeyedVectors.load_word2vec_format('/content/gdrive/My Drive/yetty/data/all/allwang.txt')
embedding_model

In [0]:
embedding_dim = len(embedding_model[next(iter(embedding_model.vocab))])
embedding_dim

100

In [0]:
embedding_matrix = np.random.rand(128, embedding_dim)
embedding_matrix

array([[0.67334653, 0.25330103, 0.58441488, ..., 0.85212844, 0.41106069,
        0.91684802],
       [0.61443696, 0.35317711, 0.3937818 , ..., 0.35112873, 0.26156577,
        0.88438597],
       [0.55450353, 0.43587813, 0.12483314, ..., 0.89894735, 0.62546321,
        0.95531268],
       ...,
       [0.37857429, 0.00348981, 0.82636624, ..., 0.90617931, 0.34458512,
        0.35262095],
       [0.13894049, 0.00141334, 0.23965882, ..., 0.53071724, 0.85242345,
        0.96849986],
       [0.38485477, 0.90038657, 0.86060933, ..., 0.08517784, 0.86481835,
        0.22881968]])

In [0]:
def precision(y_true, y_pred):	
    """Precision metric.	
    Only computes a batch-wise average of precision. Computes the precision, a
    metric for multi-label classification of how many selected items are
    relevant.
    """	
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))	
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))	
    precision = true_positives / (predicted_positives + K.epsilon())	
    return precision

In [0]:
CLASSES = 4
max_len = 50
word_size = 100

def build_model(char_size, dropout, lr, optimizer):
    model = Sequential()
    model.add(Embedding(char_size, word_size, input_length=max_len, weights=[embedding_matrix]))
    model.add(Bidirectional(LSTM(char_size, dropout=dropout, recurrent_dropout=dropout, return_sequences=False), merge_mode='sum'))
    model.add(Dropout(dropout))
    model.add((Dense(CLASSES, activation='softmax')))
    sgd = optimizers.SGD(lr=lr, momentum=0.95)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=[precision, 'accuracy'])
    return model
  
  
model = KerasClassifier(build_model)

In [0]:
def save(model):  
    timestring = "".join(str(time.time()).split("."))  ###Save each epoch with a timestamp
    model_json_name = '/content/gdrive/My Drive/yetty/data/all/resources/modelnew_{}.json'.format(timestring)
    model_json = model.model.to_json()
    with open(model_json_name, "w") as json_file:
        json_file.write(model_json)
    return model_json
  
  
def get_metrics(y_pred, y_true):  
    chosen_metrics = {
        'accuracy': metrics.accuracy_score,
        'precision' : metrics.precision_score
    }

    results = {}
    for metric_name, metric_func in chosen_metrics.items():
        try:
            if metric_name == "precision":
                inter_res = metric_func(y_pred, y_true, average=None)
            else:
                inter_res = metric_func(y_pred, y_true)
        except Exception as ex:
            inter_res = None
            print("Couldn't evaluate %s because of %s", metric_name, ex)
        results[metric_name] = inter_res
    return results


def scorer_callback(model, X, y):  
    # do all the work and return some of the metrics

    y_pred_val = model.predict(X)
    results = get_metrics(y_pred_val, y)
    model_savepath = save(model)
    return results['accuracy']

In [0]:
params_dict = {
    'lr': [0.04, 0.035],
    'char_size': [128],
    'dropout': [0.15, 0.25],
    'optimizer': ['sgd'],
}

grid = GridSearchCV(model,
                    cv=2,
                    param_grid=params_dict,
                    return_train_score=True,
                    scoring=scorer_callback)


grid_results = grid.fit(X_train,y_train)

print('Parameters of the best model: ')
print(grid_results.best_params_)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/1
16750/16750 [==============================] - 220s 13ms/step - loss: 1.2835 - precision: 0.1807 - acc: 0.3436
Epoch 1/1
16750/16750 [==============================] - 220s 13ms/step - loss: 1.2839 - precision: 0.2064 - acc: 0.3507
Epoch 1/1
16750/16750 [==============================] - 219s 13ms/step - loss: 1.2836 - precision: 0.1803 - acc: 0.3519
Epoch 1/1
16750/16750 [==============================] - 219s 13ms/step - loss: 1.2865 - precision: 0.1988 - acc: 0.3414
Epoch 1/1
16750/16750 [==============================] - 223s 13ms/step - loss: 1.3023 - precision: 0.2993 - acc: 0.3374
Epoch 1/1
16750/16750 [==============================] - 220s 13ms/step - loss: 1.3003 - precision: 0.3026 - acc: 0.3404
Epoch 1/1
16750/16750 [==============================] - 221s 13ms/step - loss: 1.3068 - precision: 0.3394 - acc: 0.3300
Epoch 1/1
16750/16750 [==============================] - 223s 13ms/step - loss: 1.3069 - precision: 0.3310 - acc: 0.3373
Epoch 1/1
33500/33500 [=========

In [0]:
grid.best_score_

0.43334328358208957

In [0]:
t = grid.best_estimator_.predict(X_test)

In [0]:
t.shape
set(t)

{0, 3}

In [0]:
u = np.argmax(l, axis=1)

In [0]:
u.shape, t.shape

((6600,), (6600,))

In [0]:
u[1]

0

In [0]:
 t[1]

0

In [0]:
(u==t).all()

True

In [0]:
model_json = grid.best_estimator_.model.to_json()
with open("/content/gdrive/My Drive/yetty/data/all/resources/grid_model.json", "w") as file:
    file.write(model_json)

In [0]:
grid.best_estimator_.model.save_weights('/content/gdrive/My Drive/yetty/data/all/resources/weights/grid_weights.h5', overwrite=True)

In [0]:
# @title
from tensorflow.keras.models import model_from_json
    
    
loaded_model = model_from_json(open('/content/gdrive/My Drive/yetty/data/all/resources/grid_model.json').read())


# load weights into new model
loaded_model.load_weights('/content/gdrive/My Drive/yetty/data/all/resources/weights/grid_weights.h5',)
print("Loaded model from disk")

loaded_model.compile(loss='categorical_crossentropy', optimizer='sgd')


Loaded model from disk


In [0]:
d = loaded_model.predict(X_test)

In [0]:
X = d
X

array([3, 3, 0, ..., 0, 0, 0])

In [0]:
c = np.argmax(d, axis=1)

In [0]:
(c == X).all()

True